In [1]:
# 1. Install necessary libraries
!pip install --upgrade google-cloud-aiplatform google-cloud-bigquery

print("--- Libraries installed ---")

--- Libraries installed ---


In [2]:
# 2. Import Libraries
import sys
import os
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from vertexai.language_models import TextEmbeddingModel
from google.cloud import bigquery

print("--- Modules imported ---")

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


--- Modules imported ---


In [3]:
# 3. Configuration

# Your Google Cloud project ID
PROJECT_ID = "qwiklabs-gcp-03-c72749959822"

# The region for your Vertex AI and BigQuery resources
LOCATION = "us-central1"

# The full BigQuery table ID in the format: project.dataset.table
TABLE_ID = "qwiklabs-gcp-03-c72749959822.genai_bootcamp.aurora_bay_faqs_embedded"

# This MUST be the same model used to create your embeddings.
# Common models: "text-embedding-004", "textembedding-gecko@003"
EMBEDDING_MODEL_NAME = "text-embedding-005"

# Model for generation
LLM_MODEL_NAME = "gemini-2.5-pro"

print(f"--- Configuration set for project: {PROJECT_ID} ---")

--- Configuration set for project: qwiklabs-gcp-03-c72749959822 ---


In [4]:
# 4. Authenticate User
# This block explicitly authenticates your user account in Colab.
try:
    from google.colab import auth
    auth.authenticate_user()
    print("--- Authentication successful ---")
except ImportError:
    print("Not in a Colab environment, assuming default credentials.")

--- Authentication successful ---


In [5]:
# 5. Initialize Vertex AI and BigQuery Clients
# We wrap this in a try/except block to catch initialization errors
print("Initializing Vertex AI and BigQuery clients...")
try:
    vertexai.init(project=PROJECT_ID, location=LOCATION)

    # Authenticate BQ PROJECT_ID
    bq_client = bigquery.Client(project=PROJECT_ID)

    embedding_model = TextEmbeddingModel.from_pretrained(EMBEDDING_MODEL_NAME)
    llm = GenerativeModel(LLM_MODEL_NAME)

    print("✅ Clients initialized successfully.")

except Exception as e:
    print(f"🚨 ERROR: Failed to initialize clients. See details below.")
    print(f"Error message: {e}")
    print("Please check your PROJECT_ID, LOCATION, and ensure APIs (Vertex AI, BigQuery) are enabled.")
    # Stop execution if clients fail
    sys.exit("Client initialization failed. Please fix the error above and restart the kernel.")

Initializing Vertex AI and BigQuery clients...
✅ Clients initialized successfully.


/usr/local/lib/python3.12/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [6]:
# 6. Chatbot Function Definition

def ask_chatbot(user_query: str, top_k: int = 3) -> str:
    """
    Performs the full RAG pipeline:
    1. Embeds the query.
    2. Retrieves context from BigQuery using VECTOR_SEARCH.
    3. Generates an answer using an LLM.
    """

    print(f"\nDEBUG: Embedding user query: '{user_query}'")

    # --- 1. RETRIEVE (Embed Query and Search BigQuery) ---
    try:
        # Generate embedding for the user's query
        query_embedding = embedding_model.get_embeddings([user_query])[0].values

        # Build the BigQuery VECTOR_SEARCH query
        sql_query = f"""
        SELECT
            base.question,
            base.answer,
            base.content,
            distance
        FROM
            VECTOR_SEARCH(
                TABLE `{TABLE_ID}`,
                'ml_generate_embedding_result',  -- The column with your embeddings
                (SELECT {query_embedding} AS ml_generate_embedding_result),
                top_k => {top_k},
                distance_type => 'COSINE'
            )
        """

        # Execute the query
        print(f"DEBUG: Running BigQuery vector search...")
        query_job = bq_client.query(sql_query)
        results = query_job.result()

        # Format the retrieved data as context
        context = ""
        for row in results:
            context += f"Source Question: {row.question}\nAnswer: {row.answer}\nContent: {row.content}\n---\n"

        if not context:
            print("DEBUG: No relevant documents found in BigQuery.")
            return "I'm sorry, I couldn't find any relevant information to answer your question."

        print(f"DEBUG: Retrieved context:\n{context}")

    except Exception as e:
        print(f"🚨 ERROR during BigQuery vector search: {e}")
        return f"Error during BigQuery vector search: {e}"

    # --- 2. GENERATE (Pass Context and Query to LLM) ---
    prompt = f"""
    You are a helpful assistant. Your task is to answer the user's question based *only*
    on the provided context. If the context does not contain the answer,
    state that you don't have that information.

    **Provided Context:**
    {context}

    **User's Question:**
    {user_query}

    **Answer:**
    """

    try:
        # Generate the final answer
        print("DEBUG: Generating answer with LLM...")
        response = llm.generate_content([Part.from_text(prompt)])
        return response.text
    except Exception as e:
        print(f"🚨 ERROR during answer generation: {e}")
        return f"Error during answer generation: {e}"

print("--- Chatbot function defined. Ready to use. ---")

--- Chatbot function defined. Ready to use. ---


In [7]:
# 7. Interactive Chat Loop
print("\n--- 🤖 Chatbot is ready ---")
print("Ask a question. Type 'quit' to exit.")

while True:
    # Get user input from the Colab prompt
    user_question = input("\nYour Question: ")

    if user_question.lower() == 'quit':
        print("\nGoodbye!")
        break

    # Get the chatbot's answer
    response = ask_chatbot(user_question)

    print("---------------------------------")
    print(f"Chatbot: {response}")
    print("---------------------------------")


--- 🤖 Chatbot is ready ---
Ask a question. Type 'quit' to exit.

Your Question: What companies operate in Aurora Bay?

DEBUG: Embedding user query: 'What companies operate in Aurora Bay?'
DEBUG: Running BigQuery vector search...
DEBUG: Retrieved context:
Source Question: What are the primary industries in Aurora Bay?
Answer: The primary industries include commercial fishing, tourism, and small-scale logging in the nearby forests.
Content: What are the primary industries in Aurora Bay?The primary industries include commercial fishing, tourism, and small-scale logging in the nearby forests.
---
Source Question: What local outdoor adventure companies operate in Aurora Bay?
Answer: North Star Excursions and Bay Explorers offer guided hikes, kayak tours, and wilderness camping experiences for visitors and residents.
Content: What local outdoor adventure companies operate in Aurora Bay?North Star Excursions and Bay Explorers offer guided hikes, kayak tours, and wilderness camping experience